## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-25-12-56-05 +0000,nypost,Zohran Mamdani transition team raises $3.7M — ...,https://nypost.com/2025/12/25/us-news/zohran-m...
1,2025-12-25-12-36-10 +0000,48hills,Screen Grabs: Triumph of the pencil-‘stached u...,https://48hills.org/2025/12/screen-grabs-trium...
2,2025-12-25-12-30-00 +0000,nypost,GOP gov hopeful Bruce Blakeman slams Hochul ov...,https://nypost.com/2025/12/25/us-news/blakeman...
3,2025-12-25-12-19-20 +0000,nypost,Returning a holiday gift? Big retail stores wi...,https://nypost.com/2025/12/25/lifestyle/retail...
4,2025-12-25-12-09-11 +0000,nyt,Kennedy Center’s Christmas Eve Jazz Show Cance...,https://www.nytimes.com/2025/12/24/us/kennedy-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2358/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
45,christmas,26
50,trump,22
32,new,11
58,year,10
374,killed,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
55,2025-12-25-06-33-11 +0000,nypost,Trump-Kennedy Center Christmas Eve concert can...,https://nypost.com/2025/12/25/us-news/trump-ke...,71
4,2025-12-25-12-09-11 +0000,nyt,Kennedy Center’s Christmas Eve Jazz Show Cance...,https://www.nytimes.com/2025/12/24/us/kennedy-...,69
90,2025-12-25-00-41-15 +0000,bbc,Where is Santa now? Trump answers kids' Christ...,https://www.bbc.com/news/videos/cqxqvd0z5gno?a...,61
136,2025-12-24-20-32-57 +0000,nypost,Pope Leo XIV calls for day of worldwide peace ...,https://nypost.com/2025/12/24/world-news/pope-...,58
108,2025-12-24-23-07-16 +0000,nypost,Passenger explodes at Alaska Airlines staff fo...,https://nypost.com/2025/12/24/us-news/passenge...,57


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
55,71,2025-12-25-06-33-11 +0000,nypost,Trump-Kennedy Center Christmas Eve concert can...,https://nypost.com/2025/12/25/us-news/trump-ke...
148,50,2025-12-24-19-07-18 +0000,nypost,Italian leader Giorgia Meloni gives year-end a...,https://nypost.com/2025/12/24/world-news/itali...
95,48,2025-12-25-00-09-16 +0000,nypost,Former Broadway child star allegedly killed by...,https://nypost.com/2025/12/24/us-news/former-b...
145,41,2025-12-24-19-26-29 +0000,nypost,DOJ unearths 1 million more possible Jeffrey E...,https://nypost.com/2025/12/24/us-news/doj-unea...
48,34,2025-12-25-08-24-14 +0000,nypost,"Powerful holiday storm lashes California, brin...",https://nypost.com/2025/12/25/us-news/southern...
136,32,2025-12-24-20-32-57 +0000,nypost,Pope Leo XIV calls for day of worldwide peace ...,https://nypost.com/2025/12/24/world-news/pope-...
107,32,2025-12-24-23-12-53 +0000,wapo,Trump-backed conservative candidate wins Hondu...,https://www.washingtonpost.com/world/2025/12/2...
26,31,2025-12-25-10-30-00 +0000,wsj,Big corporations are booming on AI and record ...,https://www.wsj.com/business/the-economic-divi...
172,25,2025-12-24-16-53-16 +0000,nypost,Afghan DC National Guard shooting suspect Rahm...,https://nypost.com/2025/12/24/us-news/afghan-d...
201,24,2025-12-24-14-54-15 +0000,nypost,Infamous sex club founder who outed Hunter Bid...,https://nypost.com/2025/12/24/us-news/infamous...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
